In [45]:
import pickle
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [14]:
with open('data/review_urls.pkl', 'rb') as f:
    review_urls = pickle.load(f)

In [17]:
page2_urls = [r+'?start=20' for r in review_urls]
page3_urls = [r+'?start=40' for r in review_urls]
page4_urls = [r+'?start=60' for r in review_urls]

In [19]:
all_urls = (page2_urls + page3_urls + page4_urls)

In [43]:
test = ['https://www.yelp.com/biz/blue-coco-new-york?start=20', 'https://www.yelp.com/biz/blue-coco-new-york?start=40', 'https://www.yelp.com/biz/blue-coco-new-york?start=60']

In [47]:
url = test[0]
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [48]:
soup

<!DOCTYPE HTML>

<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->
<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->
<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml"> <!--<![endif]-->
<head>
<script>            window.yPageStart = new Date().getTime();
</script>
<script>            var initialVisibilityState = document.webkitVisibilityState;

                yPerfTimings = [];

                ySitRepParams = {"clientIP": "104.247.38.147", "datacenter": "us-east-1", "is_internal_ip": false, "edgeStartTime": 1536157105567851, "cfRayID": null, "site

In [67]:
reviews = []
for item in soup.find_all(class_='review-content'):
    rating = float(item.find('img')['alt'].replace(' star rating',''))
    for p in item.find_all('p',{'lang':'en'}):
        for br in p.find_all('br'):
            br.replace_with('\n')
        reviews.append((rating, p.text))

In [68]:
reviews

[(3.0,
  "It is a clean place. But the salads are filled up about a third of the way and they say this is because it is chopped. When I go to Chop't or other salad places, they don't give you half a salad though?"),
 (5.0,
  "I decided to try a new place through Seamless and I'm glad I did! East Harlem doesn't have as many free delivery options as other neighborhoods, so I was glad to see a healthy option. \xa0\n\nI got the Blue Parm Chicken Sandwich (chicken cutlet, mozzarella cheese, tomato, mixed greens and mayonnaise) for $6. I also got the Antioxidant Punch Smoothie (Papaya, kale, spinach, banana and apple cider). The smoothie was 16 oz for $4 and delicious! \n\nIt was fast, friendly delivery on a rainy day! Perfect :)"),
 (4.0,
  "This is a cute spot offering healthy food options in East Harlem. I came in on a Saturday afternoon since I was in the mood for a salad. The menu is written on the wall and contains the sandwich and smoothie options, there's a salad bar where you can ju

In [38]:
def get_reviews(review_urls):
    yelp_reviews = []

    for i, url in enumerate(review_urls):
        print(i, ' of ', len(review_urls))

        try:
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')

            phone = soup.find(class_='biz-phone').text.strip().replace('(','+').replace(')','').replace('-','').replace(' ','')

            reviews = []
            for item in soup.find_all(class_='review-content'):
                for p in item.find_all('p',{'lang':'en'}):
                    for br in p.find_all('br'):
                        br.replace_with('\n')
                    reviews.append(p.text)

            rest_dict = {'phone': phone}
            rest_dict['reviews'] = reviews
            rest_dict['url'] = url

            yelp_reviews.append(rest_dict)

        except AttributeError:
            print('error at index ', i)
            continue

    return yelp_reviews

In [39]:
test_return = get_reviews(test)

0  of  3


NameError: name 'rest_dict' is not defined

In [36]:
test_return

[{'phone': '+2125342626',
  'reviews': ["It is a clean place. But the salads are filled up about a third of the way and they say this is because it is chopped. When I go to Chop't or other salad places, they don't give you half a salad though?",
   "I decided to try a new place through Seamless and I'm glad I did! East Harlem doesn't have as many free delivery options as other neighborhoods, so I was glad to see a healthy option. \xa0\n\nI got the Blue Parm Chicken Sandwich (chicken cutlet, mozzarella cheese, tomato, mixed greens and mayonnaise) for $6. I also got the Antioxidant Punch Smoothie (Papaya, kale, spinach, banana and apple cider). The smoothie was 16 oz for $4 and delicious! \n\nIt was fast, friendly delivery on a rainy day! Perfect :)",
   "This is a cute spot offering healthy food options in East Harlem. I came in on a Saturday afternoon since I was in the mood for a salad. The menu is written on the wall and contains the sandwich and smoothie options, there's a salad bar

In [40]:
len(page2_urls)

4551

In [41]:
page2_urls[366]

'https://www.yelp.com/biz/eclair-bakery-new-york?start=20'